In [36]:
import networkx as nx
import pandas as pd
import json
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

In [2]:
target=pd.read_csv("./twitch/FR/musae_FR_target.csv")

In [3]:
edges=pd.read_csv("./twitch/FR/musae_FR_edges.csv")

In [4]:
target["partner"].value_counts()

False    6141
True      410
Name: partner, dtype: int64

In [5]:
target

,id,days,mature,views,partner,new_id
0,150417538,577,True,775,False,4867
1,125723704,861,True,2585,False,3692
2,155971814,523,False,1566,False,3816
3,35832890,2234,False,9713,False,416
4,46787750,1896,True,44529,False,4502
...,...,...,...,...,...,...
6546,29196284,2061,False,3822,False,6546
6547,124327806,880,True,30313,False,2392
6548,144229380,640,True,5022,False,391
6549,131423064,736,False,2658,False,2793


In [6]:
graph_path="./twitch/FR/musae_FR_features.json"

In [7]:

# load JSON data from file
with open(graph_path, 'r') as f:
    data = json.load(f)

# create new NetworkX graph
G = nx.Graph()


In [57]:
G = nx.from_dict_of_lists(data)

# visualize the graph
#nx.draw(G, with_labels=True)

In [9]:
mat=nx.adjacency_matrix(G).toarray()

In [10]:
mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [11]:
target

,id,days,mature,views,partner,new_id
0,150417538,577,True,775,False,4867
1,125723704,861,True,2585,False,3692
2,155971814,523,False,1566,False,3816
3,35832890,2234,False,9713,False,416
4,46787750,1896,True,44529,False,4502
...,...,...,...,...,...,...
6546,29196284,2061,False,3822,False,6546
6547,124327806,880,True,30313,False,2392
6548,144229380,640,True,5022,False,391
6549,131423064,736,False,2658,False,2793


In [12]:
#Algorithme 1

In [13]:
def binary(x):
    if x==True:
        return 1
    else:
        return -1

In [14]:
target["mature"]=target["mature"].apply(binary)

In [15]:
target["partner"]=target["partner"].apply(binary)

In [16]:
unlabeled,labeled=train_test_split(target,test_size=0.8)

In [17]:
labeled

,id,days,mature,views,partner,new_id
2415,95251456,1196,1,140644,1,4440
2739,118506764,941,-1,1510,-1,389
409,117813517,951,-1,15006,-1,4928
4550,35642174,2241,1,27791,-1,2784
1451,96304477,1184,-1,306501,1,6246
...,...,...,...,...,...,...
5003,161905009,471,-1,756,-1,5819
4803,63988859,1586,-1,22834,-1,2745
1796,107372429,1058,-1,1818,-1,3230
269,68830088,1521,1,527,-1,2153


In [18]:
unlabeled

,id,days,mature,views,partner,new_id
5816,87836148,1284,-1,8165,-1,581
6136,81628563,1348,-1,36150,-1,1380
3275,28505563,2416,-1,37921,-1,5447
4066,38205619,2117,-1,6309,-1,5746
134,104731203,1073,1,18252,-1,2077
...,...,...,...,...,...,...
2840,40793626,2052,-1,5319,-1,5474
1349,47404841,1833,1,1934,-1,1098
1279,117351355,956,1,86632,1,3637
1794,81022763,1354,-1,634980,1,1527


In [19]:
import numpy as np

def label_propagation(G, Yl, max_iter=100, tol=1e-4):
    """
    Label propagation algorithm for graph G with initial labels Yl.

    Parameters:
    - G: numpy array, adjacency matrix of the graph.
    - Yl: numpy array, initial labels for some of the nodes.
    - max_iter: int, maximum number of iterations.
    - tol: float, convergence threshold.

    Returns:
    - Y: numpy array, final labels for all nodes.
    """
    D = np.diagflat(np.sum(G, axis=1)[0:,])
    

    # Compute the transition matrix.
    P = np.linalg.inv(D) @ G

    # Initialize the label vector.
    Y = np.zeros(G.shape[0])
    Y[Yl[:, 0]] = Yl[:, 1]

    # Iterate until convergence.
    for i in range(max_iter):
        Y_new = P @ Y
        Y_new[Yl[:, 0]] = Y[Yl[:, 0]]
        if np.linalg.norm(Y_new - Y) < tol:
            break
        Y = Y_new

    return Y


In [40]:
Yl=[]
for _,row in labeled.iterrows():
    Yl.append([row["new_id"],row["partner"]])

Yl_unlab=[]
for _,row in unlabeled.iterrows():
    Yl_unlab.append([row["new_id"],row["partner"]])

In [41]:
Yl=np.array(Yl)

Yl_unlab=np.array(Yl_unlab)

Y=label_propagation(mat,Yl)


In [22]:
correct=0
negpred=0
pospred=0
for i in range(0,Yl_unlab.shape[0]):
    ind=Yl_unlab[i,0]
    truelabel=Yl_unlab[i,1]
    if Y[ind]>=0 and  truelabel>=0:
        correct+=1
        pospred+=1
    if Y[ind]<=0 and  truelabel<=0:
        correct+=1
        negpred+=1

In [23]:
correct/Yl_unlab.shape[0]

0.9458015267175572

In [24]:
negpred

1238

In [25]:
pospred

1

In [26]:
unlabeled["partner"].value_counts()

-1    1238
 1      72
Name: partner, dtype: int64

In [27]:
max_iter=10000

In [28]:
tol=1e-4

In [29]:
mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [30]:
G=mat

In [31]:
D = np.diagflat(np.sum(mat, axis=1)[0:,])
print(D)

# Compute the transition matrix.
P = np.linalg.inv(D) @ mat

# Initialize the label vector.
Y = np.zeros(mat.shape[0])
Y[Yl[:, 0]] = Yl[:, 1]

# Iterate until convergence.
for i in range(max_iter):
    Y_new = P @ Y
    Y_new[Yl[:, 0]] = Y[Yl[:, 0]]
    if np.linalg.norm(Y_new - Y) < tol:
        break
    Y = Y_new

[[11  0  0 ...  0  0  0]
 [ 0 29  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  1  0]
 [ 0  0  0 ...  0  0  1]]


In [32]:
import numpy as np
from sklearn.base import clone

def iterative_classification(G, Yl, clf, max_iter=100, tol=1e-4):
    """
    Iterative classification algorithm for graph G with initial labels Yl.

    Parameters:
    - G: numpy array, adjacency matrix of the graph.
    - Yl: numpy array, initial labels for some of the nodes.
    - clf: scikit-learn classifier object, classifier to use for classification.
    - max_iter: int, maximum number of iterations.
    - tol: float, convergence threshold.

    Returns:
    - Y: numpy array, final labels for all nodes.
    """
    D = np.diagflat(np.sum(G, axis=1)[0:,])
    
    # Compute the transition matrix.
    P = np.linalg.inv(D) @ G

    # Initialize the label vector.
    Y = np.zeros(G.shape[0])
    Y[Yl[:, 0]] = Yl[:, 1]

    # Initialize feature matrix.
    X = np.zeros((G.shape[0], clf.n_features_))
    X[Yl[:, 0]] = clf.transform(G[Yl[:, 0], :])

    # Iterate until convergence.
    for i in range(max_iter):
        # Train classifier on labeled nodes.
        clf.fit(X[Yl[:, 0]], Y[Yl[:, 0]])
        
        # Predict labels for unlabeled nodes.
        Y_new = clf.predict(X[Vu, :])
        
        # Update labeled nodes.
        Y[Yl[:, 0]] = Yl[:, 1]
        X[Yl[:, 0]] = clf.transform(G[Yl[:, 0], :])
        
        # Check for convergence.
        if np.linalg.norm(Y_new - Y[Vu]) < tol:
            break
        
        # Add new labeled nodes.
        new_labeled = np.vstack((Vu[Y_new != Y[Vu]], Y_new[Y_new != Y[Vu]])).T
        Yl = np.vstack((Yl, new_labeled))
        
        # Update feature matrix.
        X[new_labeled[:, 0]] = clf.transform(G[new_labeled[:, 0], :])
        
        # Update labeled and unlabeled node sets.
        Vl = np.unique(Yl[:, 0])
        Vu = np.setdiff1d(range(G.shape[0]), Vl)
    
    return Y


In [33]:
label_to_predict=labeled["partner"]

label_to_test=unlabeled["partner"]

train=labeled.drop("partner",axis=1)

test=unlabeled.drop("partner",axis=1)

In [ ]:



param_grid = {
    'max_depth': [2, 4, 6, 8],
    'n_estimators': [50, 100, 200, 400]
}

clf = RandomForestClassifier(random_state=0)

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1,scoring="recall")
grid_search.fit(train, label_to_predict)

print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


In [38]:
clf = RandomForestClassifier(max_depth=8,n_estimators=100)

In [42]:
iterative_classification(G,Yl,clf)

AttributeError: 'RandomForestClassifier' object has no attribute 'transform'

In [46]:
# Initialize feature matrix.
X = np.zeros((G.shape[0], clf.n_classes_))
X[Yl[:, 0]] = clf.predict_proba(G[Yl[:, 0], :])

# Iterate until convergence.
for i in range(max_iter):
    # Train classifier on labeled nodes.
    clf.fit(X[Yl[:, 0]], Y[Yl[:, 0]])
    
    # Predict probability estimates for unlabeled nodes.
    Y_proba = clf.predict_proba(G[Vu, :])
    
    # Update labeled and unlabeled node features.
    X[Yl[:, 0]] = clf.predict_proba(G[Yl[:, 0], :])
    X[Vu] = Y_proba
    
    # Check for convergence.
    if np.linalg.norm(Y_proba - Y[Vu]) < tol:
        break
    
    # Add new labeled nodes.
    new_labeled = np.vstack((Vu[Y_proba.argmax(axis=1) != Y[Vu]], Y_proba[Y_proba.argmax(axis=1) != Y[Vu]])).T
    Yl = np.vstack((Yl, new_labeled))
    
    # Update labeled and unlabeled node sets.
    Vl = np.unique(Yl[:, 0])
    Vu = np.setdiff1d(range(G.shape[0]), Vl)


ValueError: Number of features of the model must match the input. Model n_features is 5 and input n_features is 8824 

In [47]:
D = np.diagflat(np.sum(G, axis=1)[0:,])

# Compute the transition matrix.
P = np.linalg.inv(D) @ G

# Initialize the label vector.
Y = np.zeros(G.shape[0])
Y[Yl[:, 0]] = Yl[:, 1]

# Initialize feature matrix.
X = np.zeros((G.shape[0], clf.n_features_))
X[Yl[:, 0]] = clf.predict_proba(G[Yl[:, 0], :]) # Use predict_proba to get class probabilities, then convert to feature matrix
X = np.concatenate([1 - X, X], axis=1) # Convert class probabilities to one-hot encoding

# Iterate until convergence.
for i in range(max_iter):
    # Train classifier on labeled nodes.
    clf.fit(X[Yl[:, 0]], Y[Yl[:, 0]])

    # Predict labels for unlabeled nodes.
    Y_new = clf.predict(X[Vu, :])

    # Update labeled nodes.
    Y[Yl[:, 0]] = Yl[:, 1]
    X[Yl[:, 0]] = clf.predict_proba(G[Yl[:, 0], :]) # Use predict_proba to get class probabilities, then convert to feature matrix
    X = np.concatenate([1 - X, X], axis=1) # Convert class probabilities to one-hot encoding

    # Check for convergence.
    if np.linalg.norm(Y_new - Y[Vu]) < tol:
        break

    # Add new labeled nodes.
    new_labeled = np.vstack((Vu[Y_new != Y[Vu]], Y_new[Y_new != Y[Vu]])).T
    Yl = np.vstack((Yl, new_labeled))

    # Update feature matrix.
    X[new_labeled[:, 0]] = clf.predict_proba(G[new_labeled[:, 0], :]) # Use predict_proba to get class probabilities, then convert to feature matrix
    X = np.concatenate([1 - X, X], axis=1) # Convert class probabilities to one-hot encoding

    # Update labeled and unlabeled node sets.
    Vl = np

ValueError: Number of features of the model must match the input. Model n_features is 5 and input n_features is 8824 

In [43]:
    D = np.diagflat(np.sum(G, axis=1)[0:,])
    
    # Compute the transition matrix.
    P = np.linalg.inv(D) @ G

    # Initialize the label vector.
    Y = np.zeros(G.shape[0])
    Y[Yl[:, 0]] = Yl[:, 1]

    # Initialize feature matrix.
    X = np.zeros((G.shape[0], clf.n_features_))
    X[Yl[:, 0]] = clf.transform(G[Yl[:, 0], :])

    # Iterate until convergence.
    for i in range(max_iter):
        # Train classifier on labeled nodes.
        clf.fit(X[Yl[:, 0]], Y[Yl[:, 0]])
        
        # Predict labels for unlabeled nodes.
        Y_new = clf.predict(X[Vu, :])
        
        # Update labeled nodes.
        Y[Yl[:, 0]] = Yl[:, 1]
        X[Yl[:, 0]] = clf.transform(G[Yl[:, 0], :])
        
        # Check for convergence.
        if np.linalg.norm(Y_new - Y[Vu]) < tol:
            break
        
        # Add new labeled nodes.
        new_labeled = np.vstack((Vu[Y_new != Y[Vu]], Y_new[Y_new != Y[Vu]])).T
        Yl = np.vstack((Yl, new_labeled))
        
        # Update feature matrix.
        X[new_labeled[:, 0]] = clf.transform(G[new_labeled[:, 0], :])
        
        # Update labeled and unlabeled node sets.
        Vl = np.unique(Yl[:, 0])
        Vu = np.setdiff1d(range(G.shape[0]), Vl)
    

AttributeError: 'RandomForestClassifier' object has no attribute 'transform'

In [45]:
X.shape

(8824, 5)

In [39]:
clf.fit(train,label_to_predict)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
pred=clf.predict(test)

In [60]:
pred

array([-1, -1, -1, ..., -1,  1, -1], dtype=int64)

In [61]:
from sklearn.metrics import accuracy_score,recall_score

print(accuracy_score(label_to_test, pred))
print(recall_score(label_to_test,pred))




0.9786259541984733
0.7692307692307693


In [51]:
unlabeled["partner"].value_counts()

-1    1232
 1      78
Name: partner, dtype: int64

In [55]:
pd.DataFrame(pred).value_counts()

-1    1242
 1      68
dtype: int64

In [60]:
with open('../facebook/1684.edges', 'r') as f:
    edges = [tuple(map(int, line.strip().split())) for line in f]


In [61]:
edges

[(2849, 3021),
 (2694, 3096),
 (2818, 2725),
 (2951, 3285),
 (3082, 2832),
 (2960, 2800),
 (2929, 3029),
 (2909, 2880),
 (2754, 2872),
 (3232, 2880),
 (3184, 2948),
 (3277, 3297),
 (3096, 2925),
 (3397, 2939),
 (3185, 3115),
 (2927, 3426),
 (3333, 3394),
 (3087, 3236),
 (3327, 2675),
 (2863, 3296),
 (3293, 3384),
 (3005, 3039),
 (3428, 3139),
 (3198, 3082),
 (3184, 3417),
 (2944, 2738),
 (3399, 2777),
 (2894, 3320),
 (2793, 2787),
 (3377, 2778),
 (3101, 3214),
 (3090, 2729),
 (3085, 2966),
 (3033, 2919),
 (2893, 2771),
 (2749, 2986),
 (2829, 2711),
 (2755, 3149),
 (3029, 2887),
 (3247, 3347),
 (3065, 2986),
 (3286, 3417),
 (3021, 3419),
 (3319, 3046),
 (2773, 2666),
 (2706, 2927),
 (2940, 2852),
 (2944, 3226),
 (2939, 3316),
 (3050, 2667),
 (3117, 3173),
 (2913, 2676),
 (2872, 3033),
 (2878, 3249),
 (2897, 3119),
 (3204, 3351),
 (2801, 3256),
 (3191, 3203),
 (2863, 3204),
 (2846, 3321),
 (3417, 2755),
 (3124, 2803),
 (3280, 3132),
 (3207, 2951),
 (2754, 2966),
 (3042, 2963),
 (3090, 29

In [62]:
# Create an empty graph
G = nx.Graph()

# Add the edges from the edge list to the graph
G.add_edges_from(edges)


In [63]:
new_matrix=nx.adjacency_matrix(G)

In [64]:
Graph=new_matrix.toarray()

In [65]:
Graph.shape

(786, 786)

In [66]:
Graph

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [53]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Create a random feature matrix and label vector
X = np.random.rand(100, 10)
y = np.random.randint(0, 2, size=100)

# Create a random forest classifier and fit it to the data
clf = RandomForestClassifier()
clf.fit(X, y)

# Compute the feature importances
importances = clf.feature_importances_


In [55]:
X.shape

(100, 10)

In [54]:
importances

array([0.09176099, 0.07446771, 0.10436013, 0.11417346, 0.10954563,
       0.1028026 , 0.09912925, 0.09672676, 0.11973828, 0.08729519])

In [ ]:
import numpy as np

# G is the graph, Y is the label vector
# A is the multiclass learning algorithm

# Initialize the classifier
X = np.zeros((len(G.nodes), num_features))  # construct feature matrix
clf = A(X, Y)  # initialize the classifier

# Iterative classification
for i in range(num_iterations):
    # Generate ordering O over nodes of G
    nodes = list(G.nodes)
    np.random.shuffle(nodes)  # shuffle nodes randomly
    
    # Iterate over nodes in O
    for node in nodes:
        # Compute conditional probabilities for each label
        xi = np.concatenate([node_features[node], neighbor_features[node]])  # construct feature vector for node
        probas = clf.predict_proba(xi.reshape(1, -1))[0]  # compute probability vector
        
        # Sample label for node
        label = np.random.choice(labels, p=probas)  # sample label from probability distribution
        
        # Submit training example to A
        neighborhood = G.neighbors(node)  # get neighbors of node
        neighbor_features = [node_features[n] for n in neighborhood]  # get feature vectors for neighbors
        X_node = np.concatenate([xi, neighbor_features], axis=0)  # construct feature matrix for node and its neighbors
        Y_node = label  # label for node
        clf.partial_fit(X_node.reshape(1, -1), [Y_node])  # update classifier with new training example


In [ ]:
#NOUVEAU CODE

In [ ]:
### Premier test : 10% des noeuds labélisés, choix des noeuds random

In [ ]:
#Comment on choisis nos points de départs : il y a un vrai travaille à faire la dessus.

In [195]:
unlabeled,labeled=train_test_split(target[["days","views","mature","new_id","partner"]],test_size=0.01)

In [196]:
print(target["partner"].value_counts())

-1    6141
 1     410
Name: partner, dtype: int64


In [197]:
print(unlabeled["partner"].value_counts(),labeled["partner"].value_counts())

-1    6080
 1     405
Name: partner, dtype: int64 -1    61
 1     5
Name: partner, dtype: int64


In [198]:
from sklearn.metrics import *

In [199]:
#G is the complete graph
#train_temp,test_temp=train_test_split(unlabeled)

while unlabeled.shape[0]>0:

    Y_temp=labeled["partner"]
    graph_lab=G.subgraph([n for n in labeled["new_id"]])
    #Y_test=test_temp["partner"]
    #Iteration 0
    param_grid = {
        'max_depth': [2, 4, 6, 8],
        'n_estimators': [50, 100, 200, 400]
    }

    clf = RandomForestClassifier(random_state=0)

    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1,scoring="recall")
    grid_search.fit(labeled.drop(columns=["new_id"]),Y_temp)

    print("Best hyperparameters:", grid_search.best_params_)
    print("Best score:", grid_search.best_score_)

    clf.set_params(**grid_search.best_params_)

    clf.fit(labeled.drop(columns=["partner","new_id"]),Y_temp)

    nodes_with_edge_to_subgraph = [n for n in G.nodes() if n not in graph_lab and any([n in G.neighbors(subgraph_node) for subgraph_node in graph_lab])]
    new_ids=[int(n) for n in nodes_with_edge_to_subgraph]

    to_label = unlabeled[unlabeled["new_id"].isin(new_ids)]

    y_test=to_label["partner"]

    y_pred=clf.predict(to_label.drop(columns=["partner","new_id"]))
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    to_label["partner"]=y_pred

    print(accuracy,precision,recall,f1)

    unlabeled = unlabeled.drop(to_label.index)
    labeled = pd.concat([labeled, to_label], axis=0)



    
    


Best hyperparameters: {'max_depth': 2, 'n_estimators': 50}
Best score: 1.0


C:\Users\33750\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.9288702928870293 0.9202025694431645 0.9288702928870293 0.9185758676847393
Best hyperparameters: {'max_depth': 2, 'n_estimators': 50}
Best score: 1.0


C:\Users\33750\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.9540412044374009 0.9540162478785255 0.9540412044374009 0.9415734154656172
Best hyperparameters: {'max_depth': 2, 'n_estimators': 50}
Best score: 1.0


C:\Users\33750\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.9461931665321496 0.9442393560154285 0.9461931665321496 0.9268466034518588
Best hyperparameters: {'max_depth': 2, 'n_estimators': 50}
Best score: 1.0
1.0 1.0 1.0 1.0


In [200]:
labeled

,days,views,mature,new_id,partner
6440,801,35916,1,1192,-1
2780,1060,10642,-1,2743,-1
6543,1226,402,-1,323,-1
1304,1677,913,1,6002,-1
2531,1847,22365,-1,2206,-1
...,...,...,...,...,...
4201,1787,22613,-1,2796,-1
2247,894,4530,1,1331,-1
3021,1601,1126,-1,1606,-1
301,2194,592,-1,4119,-1


In [201]:
target

,id,days,mature,views,partner,new_id
0,150417538,577,1,775,-1,4867
1,125723704,861,1,2585,-1,3692
2,155971814,523,-1,1566,-1,3816
3,35832890,2234,-1,9713,-1,416
4,46787750,1896,1,44529,-1,4502
...,...,...,...,...,...,...
6546,29196284,2061,-1,3822,-1,6546
6547,124327806,880,1,30313,-1,2392
6548,144229380,640,1,5022,-1,391
6549,131423064,736,-1,2658,-1,2793


In [202]:
accuracy = accuracy_score(target["partner"], labeled["partner"])
precision = precision_score(target["partner"], labeled["partner"], average='weighted')
recall = recall_score(target["partner"], labeled["partner"], average='weighted')
f1 = f1_score(target["partner"], labeled["partner"], average='weighted')

In [203]:
print(accuracy,precision,recall,f1)

0.9235231262402687 0.8787185073384991 0.9235231262402687 0.9003866555963747


In [179]:
labeled["partner"].value_counts()

-1    6251
 1     300
Name: partner, dtype: int64

In [180]:
target["partner"].value_counts()

-1    6141
 1     410
Name: partner, dtype: int64

In [163]:
to_label["partner"]=y_pred

C:\Users\33750\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [165]:
to_label["partner"]==y_pred

5856    True
5798    True
6326    True
6170    True
6003    True
        ... 
6275    True
3920    True
1780    True
2141    True
3588    True
Name: partner, Length: 3845, dtype: bool

In [162]:
y_pred

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [152]:
pd.DataFrame(result).value_counts()

-1    3665
 1     180
dtype: int64

In [154]:
to_label[["partner"]].value_counts()

partner
-1         3604
 1          241
dtype: int64

In [ ]:
result

In [136]:
to_label

,days,views,mature,new_id,partner
5856,784,1679,1,5032,-1
5798,2443,1214,-1,5293,-1
6326,1545,21647,1,5850,-1
6170,1369,17117,-1,5586,-1
6003,884,527,-1,1096,-1
...,...,...,...,...,...
6275,2352,193347,-1,2833,-1
3920,1498,1381470,1,3905,1
1780,2084,11136,1,4278,-1
2141,610,10857,1,466,-1


In [132]:
[int(n) for n in nodes_with_edge_to_subgraph]

[3032,
 4032,
 3945,
 949,
 5137,
 1803,
 2468,
 2811,
 1689,
 6077,
 5905,
 5571,
 3636,
 349,
 3867,
 1234,
 5310,
 4677,
 4619,
 1508,
 2901,
 1068,
 5693,
 6291,
 2475,
 4047,
 3377,
 4428,
 4648,
 6377,
 3321,
 1619,
 1295,
 5531,
 4852,
 181,
 5283,
 1434,
 1518,
 3744,
 1136,
 1947,
 3088,
 1022,
 6133,
 3200,
 2554,
 3257,
 3403,
 3812,
 5143,
 3242,
 2299,
 2208,
 1595,
 3689,
 1972,
 4525,
 2221,
 420,
 5594,
 5607,
 684,
 5410,
 819,
 729,
 6340,
 1816,
 5939,
 3642,
 5182,
 3774,
 3151,
 1600,
 2419,
 4481,
 4025,
 5859,
 18,
 2336,
 6515,
 3074,
 322,
 6392,
 2039,
 4946,
 6384,
 3306,
 3384,
 34,
 2250,
 3147,
 494,
 5242,
 1476,
 4420,
 4763,
 995,
 969,
 6254,
 4576,
 5196,
 3876,
 3047,
 5742,
 6021,
 1709,
 511,
 2726,
 6506,
 1357,
 4504,
 2587,
 5185,
 3395,
 6189,
 4774,
 1494,
 1650,
 2845,
 2315,
 1326,
 3037,
 6363,
 4761,
 4813,
 4553,
 1263,
 3504,
 2285,
 4955,
 2426,
 5572,
 3372,
 4265,
 4704,
 4304,
 6061,
 4328,
 5204,
 2500,
 4480,
 6523,
 1639,
 1786,
 

In [130]:
print(new_line)

      days  views  mature  new_id  partner
2356  1845   4369      -1     836       -1
